# Study the MSE

So the MSE that Majid is refering to is the MSE obtained by regressing the absorbances against the concentrations at the wavelength of greatest variance in absorbances.

Majid's hypothesis is that: "by incorporating more data than just the absorbances at the single wavelength of greatest variance, we can significantly reduce the MSE.

To test this I will:
1. calculate what the MSE is at the wavelength of highest variance
2. test increasingly larger windows around the wavelength of highest variance
   1. fix additional points to 4
   2. test 10 nm around wavelength of highest variance
   3. test 20 nm " " " " "
   4. test 40 nm " " " " "
3. with the optimal window size from above, test different densities of input data
   1. 4 points (ctrl)
   2. 8 points
   3. 16 points
   4. 32 points
4. Note: for all modeling I will use a L1-penalized linear regression model (LASSO)

# load imports and data

In [57]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
# use lasso from sklearn
from sklearn.linear_model import Lasso

In [19]:
# load the data from the previous part
data = pd.read_csv('../p01_extract_data_from_fig/colorimetry_data_PS_extracted.csv')
data.rename(columns={'Wavelength (nm)':"wavelength",'Smoothed Absorbance':"sm_absorbance"}, inplace=True)

# Determine Wavelength of highest Variance and fit model

In [20]:
wv_highest_variance = None
for wv in np.unique(data['wavelength']):
    df = data[data['wavelength'] == wv]
    if wv_highest_variance is None or df['sm_absorbance'].var() > wv_highest_variance['sm_absorbance'].var():
        wv_highest_variance = df
print(wv_highest_variance['wavelength'].values[0],wv_highest_variance['sm_absorbance'].var())

652.1784232365146 0.003993509394281987


In [15]:
wv_highest_variance

,Wavelength (nm),Absorbance,Color,Concentration,Smoothed Absorbance,webcolor
195,652.178423,0.284016,black,0,0.282795,#000000
578,652.178423,0.275820,red,5,0.275289,#FF0001
961,652.178423,0.240574,blue,10,0.239817,#0D0CF9
1344,652.178423,0.224180,green,20,0.224142,#017F03
1727,652.178423,0.191393,pink,30,0.190823,#FF02FF
2110,652.178423,0.159426,magenta,40,0.159388,#8604B4
2493,652.178423,0.144672,navy,50,0.144475,#0203A6
2876,652.178423,0.128279,orange,60,0.128354,#FF870C
3259,652.178423,0.123361,purple,80,0.122924,#9011EE
3642,652.178423,0.118443,salmon,100,0.118463,#FF0381


In [21]:
# fit an OLS model with absorbance as the dependent variable and the concentration as the independent variable
# use R model syntax
model = sm.OLS.from_formula('Concentration ~ sm_absorbance', data=wv_highest_variance)
results = model.fit()
# print the MSE of the model
print(results.mse_resid)
results.summary()

168.79064786196122


c:\Users\craut\.virtualenvs\colorimetry_ML-6CtRR2jE\lib\site-packages\scipy\stats\_axis_nan_policy.py:430: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=10 observations were given.
  return hypotest_fun_in(*args, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Concentration   R-squared:                       0.864
Model:                            OLS   Adj. R-squared:                  0.847
Method:                 Least Squares   F-statistic:                     50.79
Date:                Mon, 10 Feb 2025   Prob (F-statistic):           9.94e-05
Time:                        23:45:12   Log-Likelihood:                -38.717
No. Observations:                  10   AIC:                             81.43
Df Residuals:                       8   BIC:                             82.04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       131.6291     13.565      9.704      0.000     100.348     162.910
sm_absorbance  -488.3677     68.529     -7.126      0.000    -646.396    -330.339
==============================================================================
Omnibus:                        3.152   Durbin-Watson:                   0.635
Prob(Omnibus):                  0.207   Jarque-Bera (JB):                1.323
Skew:                           0.891   Prob(JB):                        0.516
Kurtosis:                       2.997   Cond. No.                         17.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Test different windows of 4 surrounding points

In [37]:
# first establish approximately how many wavelengths in the data correspond to ~10 nm intervals
wavelengths = sorted(np.unique(data['wavelength']))
wavelengths[19] - wavelengths[0]

np.float64(9.854771784232412)

In [71]:
# so about every 19 wavelength records corresponds to a 10 nm interval
conversion_factor = 19
dists = [1,2,4]
# get the index corresponding to the wv_highest_variance
idx = np.where(wavelengths == wv_highest_variance['wavelength'].values[0])[0][0]
for dist in dists:
    wavelengths_to_use = wavelengths[idx-(2*dist*conversion_factor):idx+(2*dist*conversion_factor)+1:(dist*conversion_factor)]
    print(wavelengths_to_use)
    df = pd.DataFrame(data[data['wavelength'].isin(wavelengths_to_use)])
    df["wavelength"] = df["wavelength"].round().astype(int)
    # convert the data to wide format
    df = df.pivot(index='Concentration', columns='wavelength', values='sm_absorbance')
    rename_dict = {c:f"abs_wv_{c}" for c in df.columns}
    df = df.rename(columns=rename_dict).reset_index()
    # fit a lasso model with the data from the selected wavelengths
    # using sklearn
    X = df.drop(columns=['Concentration'])
    # add a constant column to the data
    X = sm.add_constant(X)
    y = df['Concentration']
    lasso = Lasso(alpha=0.005)
    lasso.fit(X, y)
    # get the MSE of the model
    y_pred = lasso.predict(X)
    mse = np.mean((y - y_pred)**2)
    print(mse)
    # get the number of non-zero coefficients
    print(np.sum(lasso.coef_ != 0))
    
    

[np.float64(632.4688796680498), np.float64(642.3236514522822), np.float64(652.1784232365146), np.float64(662.0331950207469), np.float64(671.8879668049793)]
126.34437560878428
1
[np.float64(612.7593360995851), np.float64(632.4688796680498), np.float64(652.1784232365146), np.float64(671.8879668049793), np.float64(691.597510373444)]
118.94977957056707
2
[np.float64(573.3402489626556), np.float64(612.7593360995851), np.float64(652.1784232365146), np.float64(691.597510373444), np.float64(731.0165975103735)]
116.33852509509659
2


In [69]:
lasso.coef_

array([   0.        , -280.40057676, -550.26011551,   -0.        ,
          0.        ,   -0.        ])